<a href="https://colab.research.google.com/github/ghasemieh/Duplicate-Bug-Identification-System/blob/master/Duplicate_BugReport_V7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import progressbar as pb
import time
import warnings
import text_processing as tp
warnings.filterwarnings('ignore')
# data = pd.read_csv('https://zenodo.org/record/400614/files/apache.csv?download=1',sep=',')

In [0]:
def remove_nan(df):
    print('Remove NA')
    blanks = []
    print("Before removing the NaN:")
    print(df.isnull().sum())
    for x in df.itertuples():
        if type(x.short_desc)!=str: # detect the NaN
            blanks.append(x.Index)
        elif not x.short_desc: # detect empty string
            blanks.append(x.Index)
    df.drop(blanks,inplace=True)
    print("\nAfter removing the NaN:\n",df.isnull().sum())
    
# remove_nan(data)

In [0]:
# Preprocessing the short_desc
# processed_data_df = tp.text_preprocessing(data,"short_desc")
# processed_data_df.to_csv('~/Desktop/Google-Drive/Colab Notebooks/processed_data_df.csv',index=False)
# processed_data_df.head()

In [0]:
processed_data_df = pd.read_csv('~/Desktop/Google-Drive/Colab Notebooks/processed_data_df.csv',sep=',')
# remove_nan(processed_data_df)

In [0]:
# Model-1: Similarity Score - Word2vec -------------------------------------------------------------------------
import pandas as pd
import spacy
import time
import progressbar as pb
nlp = spacy.load('en_core_web_lg')

execution_count_word2vec = 0
processed_data_nlp_df = []

# Convert short_desc str to nlp format to fasten the computation process 
def word2vec_preprocess(df):
    print('Convert short_desc str to nlp format')
    sample_size = len(df)
    progress = pb.ProgressBar(maxval = sample_size).start()
    progvar  = 0
    processed_data_nlp = []

    for tup in df.itertuples():
        processed_data_nlp.append((tup.id,tup.product,nlp(tup.short_desc_processed))) 
        progress.update(progvar + 1)
        progvar += 1
    
    global processed_data_nlp_df
    processed_data_nlp_df = pd.DataFrame(processed_data_nlp, columns=['id','product','short_desc_processed'])
    global execution_count_word2vec
    execution_count_word2vec += 1

# Calculate the cosine similarity score
def word2vec_similarity(id,df):
    if execution_count_word2vec == 0:
        word2vec_preprocess(df)
#     print('Calculate the word2vec similarity score')
#     start_time  = time.time()
#     sample_size = len(processed_data_nlp_df)
#     progress    = pb.ProgressBar(maxval=sample_size).start()
#     progvar     = 0
    similarities_score_list = []
    
    product_main = processed_data_nlp_df.loc[lambda df: df['id'] == id,'product'].array[0]
    short_desc_processed_main = processed_data_nlp_df.loc[lambda df: df['id'] == id,'short_desc_processed'].array[0]
    
    for doc in processed_data_nlp_df.itertuples():
        product_other = processed_data_nlp_df.loc[lambda df: df['id'] == doc.id,'product'].array[0]
        if product_main == product_other:
            similarity_score = doc.short_desc_processed.similarity(short_desc_processed_main)
            similarities_score_list.append((doc.id,similarity_score))
#         progress.update(progvar + 1)
#         progvar += 1
#     progress = pb.ProgressBar(maxval=sample_size).finish()

    #convert to dataframe
    word2vec_similarities_score_df = pd.DataFrame(similarities_score_list, columns=['id','word2vec_score'] )
    word2vec_similarities_score_df = word2vec_similarities_score_df.reset_index(drop=True)
    # show the time of process
#     print("-word2vec- %s seconds ---" % (time.time() - start_time)) 
    return word2vec_similarities_score_df

In [0]:
# Model-2: Similarity Score - TF-idf ----------------------------------------------------------------------------
import pandas as pd
import time
import progressbar as pb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

tfidf_cosine_similarities = []
execution_count_tfidf = 0

def tfidf_preprocess(df):
    X_train = df['short_desc_processed']
    print('TF-idf Vectorization and similarity score computation')
    # Vectorization
    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform(X_train)
    # Calculate the cosine similarity score
    global tfidf_cosine_similarities
    tfidf_cosine_similarities = linear_kernel(tfidf)
    global shape_tfidf
    shape_tfidf = tfidf_cosine_similarities.shape[0]
    print('TF-idf preprocess done')
    global execution_count_tfidf
    execution_count_tfidf += 1

def tfidf_similarities(id,df):
    if execution_count_tfidf == 0:
        tfidf_preprocess(df)
#     print('Calculate the TF-idf similarity score')
#     start_time   = time.time()
#     sample_size  = len(df)
#     progress     = pb.ProgressBar(maxval = sample_size).start()
#     progvar      = 0
    index_main   = df.loc[lambda df: df['id'] == id].index.array[0]
    product_main = df.loc[lambda df: df['id'] == id,'product'].array[0]
    tfidf_cosine_similarities_list = []
    for index_other in range(shape_tfidf):
        id_other      = df.iloc[index_other]['id']
        product_other = df.iloc[index_other]['product']
        if product_main == product_other:
            tfidf_cosine_similarities_list.append([id_other,tfidf_cosine_similarities[index_main,index_other]])
#         progress.update(progvar + 1)
#         progvar += 1
#     progress = pb.ProgressBar(maxval=sample_size).finish()

    #Conver to dataframe
    tfidf_cosine_similarities_score_df = pd.DataFrame(tfidf_cosine_similarities_list, columns=['id','tfidf_score'])
    tfidf_cosine_similarities_score_df = tfidf_cosine_similarities_score_df.reset_index(drop=True)
    # show the time of process
#     print("-TF-idf- %s seconds ---" % (time.time() - start_time)) 
    return tfidf_cosine_similarities_score_df

In [0]:
# Model-3: Similarity Score - BM24F -----------------------------------------------------------------------------
import pandas as pd
import time
import progressbar as pb
from rank_bm25 import BM25Okapi

processed_corpus_list = []
bm25 = []
execution_count_bm25 = 0

# preprocess - tokenize the short_desc to token
def bm24_preprocess(df):
    print('preprocess - tokenize the short_desc to token')
    sample_size = len(df)
    global processed_corpus_list
    processed_corpus_list = []
    for x in df.itertuples():
        short_desc_splited = x.short_desc_processed.split(" ")
        processed_corpus_list.append(short_desc_splited)
    # Create a MB24 Object with the corpus
    global bm25
    bm25 = BM25Okapi(processed_corpus_list)
    global execution_count_bm25
    execution_count_bm25 += 1
    
# Calculate the similarity score
def bm24_similarity(id,df):
    if execution_count_bm25 == 0:
        bm24_preprocess(df)
#     print('Calculate the BM24 similarity score')
#     start_time    = time.time()
#     sample_size   = len(df)
#     progress      = pb.ProgressBar(maxval = sample_size).start()
#     progvar       = 0
    index_main    = df.loc[lambda df: df['id'] == id].index.array[0]
    product_main  = df.loc[lambda df: df['id'] == id,'product'].array[0]
    query         = processed_corpus_list[index_main]
    doc_scores    = bm25.get_scores(query)
    doc_scores_df = pd.DataFrame(doc_scores, columns=['bm24_score'])
    
    # add id to the score list and remove unsimiliar product
    blanks = []
    for x in doc_scores_df.itertuples():
        id_other      = df.iloc[x.Index]['id']
        product_other = df.iloc[x.Index]['product']
        # add id to the score list
        doc_scores_df.loc[x.Index,'id'] = id_other
        if product_main != product_other:
            blanks.append(x.Index)    
#         progress.update(progvar + 1)
#         progvar += 1
    doc_scores_df.drop(blanks,inplace=True)   
    doc_scores_df = doc_scores_df.reset_index(drop = True) 
    # show the time of process
#     print("-bm24- %s seconds ---" % (time.time() - start_time)) 
    return doc_scores_df

Assessment the Model Accuracy

In [0]:
# Assessment the Model Accuracy -------------------------------------------------------------------------------
# duplicate_df = pd.read_csv('https://zenodo.org/record/400614/files/apache.relations.csv?download=1',sep=',')
# duplicate_df.to_csv('~/Desktop/Google-Drive/Colab Notebooks/duplicate_df.csv',index=False)
duplicate_df = pd.read_csv('~/Desktop/Google-Drive/Colab Notebooks/duplicate_df.csv',sep=',')

In [0]:
# Calculate the similarity scores and return the first n top scoes
def similarity_score(id, df,top_n):
    word2vec_similarity_df = word2vec_similarity(id, df).sort_values(by=['word2vec_score'],ascending=False).head(top_n)
    tfidf_similarity_df    = tfidf_similarities(id, df).sort_values(by=['tfidf_score'],ascending=False).head(top_n)
    bm24_similarity_df     = bm24_similarity(id, df).sort_values(by=['bm24_score'],ascending=False).head(top_n)    
    return word2vec_similarity_df, tfidf_similarity_df, bm24_similarity_df

In [0]:
# Calculate the similarity score for the reported duplicated bug report
import progressbar as pb
import time
start_time  = time.time()
sample_size = len(duplicate_df)
progress    = pb.ProgressBar(maxval = sample_size).start()
progvar     = 0
duplicated_similarity_score_list = []

for tup in duplicate_df.itertuples():
    word2vec_similarity_df, tfidf_similarity_df, bm24_similarity_df = similarity_score(tup.id, processed_data_df,20)
    duplicated_similarity_score_list.append([tup.id,tup.dup,word2vec_similarity_df,tfidf_similarity_df,bm24_similarity_df])
    progress.update(progvar + 1)
    progvar += 1
    
print("-Total- %s seconds ---" % (time.time() - start_time))

 13% (68 of 500) |###                    | Elapsed Time: 1:05:56 ETA:   6:57:36

In [0]:
import pickle
with open("duplicated_similarity_score_list.txt", "wb") as fp:   #Pickling
    pickle.dump(duplicated_similarity_score_list, fp)

# with open("duplicated_similarity_score_list.txt", "rb") as fp:   # Unpickling
#     l = pickle.load(fp)